#### -- [INIT] -- 
***

##### **Import Libraries and Init Globals**
- `on_awake   ` (init_game => screen, refresh, load assets ... etc)
- `on_update  ` (game_loop => handle events, input, draws ..etc )
- `on_exit    ` (close_app => clear memory and shut down cpu/sys ... etc)

In [1]:
# `numpy` library random which is FASTER for generating large numbers of random values
from numpy import random
from typing import NamedTuple
import pygame
import sys
import os

# System Context will be init on awake
class SystemContext (NamedTuple):
    screen :pygame.Surface
    clock  :pygame.time.Clock

class RenderContext(NamedTuple):
    WIDTH :int
    HEIGHT :int
    BACKGROUND_COLOR :tuple
    FPS :int

class AssetContext(NamedTuple):
    ASSET_FOLDER_PATH :str

# Statically define CONTEXT CONSTANTS
RENDER_CONTEXT = RenderContext(
    640, # RESOLUTION_WIDTH
    480, # RESOLUTION_HEIGHT
    (1, 0, 0), # BACKGROUND_COLOR
    5 # FPS
)

ASSET_CONTEXT = AssetContext(
    ASSET_FOLDER_PATH = os.path.join('assets','images')
)

class BaseEntity:
    def __init__(self):
        pass
    def update(self):
        pass

class DrawEntity(BaseEntity):
    def __init__(self, x,y, shape, color = (255, 255, 255)):
        super().__init__()
        self.x = x
        self.y = y
        self.shape = shape
        self.color = color

    def draw(self, screen : pygame.Surface):
        pygame.draw.polygon(
            screen, 
            self.color, 
            [(self.x+px, self.y+py) for px,py in self.shape['points']], 
            1
        )

class GameEntity(DrawEntity):
    def __init__(self, x,y, shape, scale = 1, rotation =0, speed = 0):
        super().__init__(x,y, shape)
        self.vx = 0
        self.vy = 0
        self.rotation = rotation
        self.scale = scale
        self.speed = speed

    def update(self):
        super().update()
        # Update the position based on velocity
        self.x = (self.x + self.vx) % RENDER_CONTEXT.WIDTH
        self.y = (self.y + self.vy) % RENDER_CONTEXT.HEIGHT
        # Update the rotation
        self.rotation = (self.rotation + 1) % 360

def on_awake() -> SystemContext:
    ''' 
    Initialize pygame
    - SYS_SCREEN
    - SYS_CLOCK
    '''
    # Initialize pygame
    pygame.init()
    # Initialize the screen
    screen = pygame.display.set_mode(
        # @audit-ok 💨 : Use `HWSURFACE` and `DOUBLEBUF` for better performance
        (RENDER_CONTEXT.WIDTH, RENDER_CONTEXT.HEIGHT),
        pygame.HWSURFACE | pygame.DOUBLEBUF)
    # Initialize the clock
    # @audit-ok 💨 : Cap the frame rate else loop will run at system allows and 
    # # will lead to :
    # - high CPU usage
    # - inconsistent performance across different devices. 
    clock = pygame.time.Clock()

    # Set the title and icon (optional)
    pygame.display.set_caption('Asteroids_TON_Edition')
    icon = pygame.image.load(
        os.path.join(ASSET_CONTEXT.ASSET_FOLDER_PATH, 'app_icon.jpg')
    ).convert_alpha()
    pygame.display.set_icon(icon)
    return SystemContext(screen, clock)

def on_exit() -> None :
    ''' 
    Handle exit event
    '''
    pygame.quit()
    sys.exit()

# Game Loop
def game_loop(context_sys : SystemContext, assets : list) -> None:
    ''' 
    Game loop
    '''
    while True:
        try:
            # Set the frame rate
            context_sys.clock.tick(RENDER_CONTEXT.FPS)
            # Event Handling
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    on_exit()
                if event.type == pygame.KEYUP:
                    if event.key == pygame.K_ESCAPE:
                        on_exit()
            # Update game state
            # Draw game objects
            # Update the display
            context_sys.screen.fill(RENDER_CONTEXT.BACKGROUND_COLOR)
            for asset in assets:
                asset.update()
                asset.draw(context_sys.screen)
            pygame.display.flip()
            
        except SystemExit:
            break


pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### -- [ENTITIES] --

##### **Asteroid**

In [2]:
class Asteroid(GameEntity):
    def __init__(self, x, y, shape):
        super().__init__(x, y, shape)
        self.vx = random.randint(-2, 2)
        self.vy = random.randint(-2, 2)
        self.rotation = random.randint(0, 359)

##### **UFO**

In [3]:
class UFO(GameEntity):
    def __init__(self, x, y, shape):
        super().__init__(x, y, shape)
        self.vx = random.randint(-8, 8)
        self.vy = random.randint(-8, 8)
        self.rotation = 0
        self.color = (0, 255, 0)

#### --[Asset Package] --

##### **Generate**

In [4]:
import math

def generate_circle_points(size, num_points):
    points = []
    angle = 2 * math.pi/num_points

    for i in range(num_points):
        x = size * math.cos(angle * i)
        y = size * math.sin(angle * i)
        points.append((x,y))

    return points

def generate_asteroid_points(size, num_points):
    points = []
    angle = 2 * math.pi/num_points

    for i in range(num_points):
        length = random.randint(int(size * 0.5), size)
        x = length * math.cos(angle * i)
        y = length * math.sin(angle * i)
        points.append((x,y))

    return points

##### **Insantiate**

In [5]:
def asset_instantiate() -> list:
    assets = []
    # Instantiate UFO
    ufo_points = generate_circle_points(32, 32)
    assets.append(UFO(0,0, {'points':ufo_points}))
    # Instantiate asteroids
    for i in range(5):
        x = random.randint(0, 160)
        y = random.randint(0, 120)
        # random_scale = random.randint(16, 64)
        # img = pygame.transform.scale(img_asteroid, (random_scale, random_scale))
        asteroid_points = generate_asteroid_points(64, 12)
        assets.append(Asteroid(x,y, {'points':asteroid_points}))

    return assets

#### -- [GAME_LOOP] --

- instantiate entities
- update frames in main ()

#### **Main**

In [6]:
def main() -> None:
    context  = on_awake()
    assets = asset_instantiate()
    game_loop(context, assets)
    
# Only run if this is being executed directly
# - prevents the code from being run when imported
# @note 🧠 : __name__ is built-in variable in Python that represents :
# - the name of the current module 
# - When a script is run, Python sets the `__name__` variable to `'__main__'` 
# for the script being executed
if __name__ == '__main__':
    main()

#### -- [FOOTNOTES] --
***

Explain the `difference` between adding the `try and exception catching` above vs  
not doing a catch from `memory` and `performance` perspective
***

Adding a `try` and `except` block around your code introduces a minimal overhead  
in terms of memory and performance.  In general, the cost of using exception  
handling is low when no exceptions are raised.  

However, when an exception occurs : 
- the traceback needs to be generated  
- and the `interpreter has to search the exception hierarchy` to find the  
appropriate `except` block, which can be more `expensive`.

Here's a comparison between using exception handling and not using it in the  
context of the Pygame example we've discussed:

**Without exception handling:**

When you don't catch the `SystemExit` exception, the kernel running the Jupyter  
Notebook will display the message : 
- `An exception has occurred, use %tb to see the full traceback. SystemExit`  
whenever the game is terminated. 

This is because : 

- the `SystemExit` exception is raised, 
- caught by the notebook's kernel, 
- and displayed as output. 

The game loop will still exit as expected, and the exception handling overhead  
is minimal. However, you'll see the traceback message every time the game is  
closed, which might be `undesirable from a user experience standpoint`.

**With exception handling:**

When you include a `try` and `except` block to catch the `SystemExit` exception,  
you introduce a small overhead associated with the exception handling mechanism.  

However, this overhead is generally negligible in the context of a game loop.  
When the game is terminated, the `SystemExit` exception is raised and caught  
by the `except` block, allowing the loop to exit silently without displaying  
the traceback message. This can provide a cleaner user experience.

In summary, the difference in memory and performance between using exception  
handling and not using it in this case is minimal. The primary reason to include  
exception handling in this scenario is to improve the user experience by  
preventing the traceback message from being displayed when the game is terminated.

Explain the `trade off` between declaring variable as `global` vs passing as  
`reference` with respect to `memory and performance`
***

When deciding whether to use a global variable or pass a variable by reference,  
there are trade-offs to consider regarding memory, performance, and general  
programming practices.

**Global variables:**

Pros:

- Easy to access from any part of the code without needing to pass them as  
arguments
- Can help reduce the number of arguments a function needs, making the function  
signature simpler.

Cons:

- Can lead to unexpected behavior and bugs, as any part of the code can modify  
the global variable.
- Makes code harder to understand and maintain, as it's not clear which  
functions rely on or modify the global variable.
- Less modular and harder to reuse functions in other programs, since they  
depend on the global variable.

**Passing by reference:**

Pros:

- Makes code more modular, as functions explicitly state in their signature  
what variables they need.
- Easier to understand, maintain, and reuse, as it's clear which functions use  
or modify the variable.
- Better encapsulation, as each function deals only with the data it needs.

Cons:

- Can increase the number of function arguments, making the function signature  
more complex.
- Slightly more complex to work with, as you need to pass the variable explicitly.  

Regarding `memory and performance`, there is generally `not a significant difference`  
between using global variables and passing variables by reference.  
In both cases, the memory used for the variable is allocated once and accessed  
by reference, so there is no additional memory overhead. The performance  
depends on the specific use case and how well the code is optimized.

However, it is worth mentioning that using` global variables` can lead to  
`less maintainable code`, which may indirectly affect performance over time, as  
it becomes more difficult to optimize and debug the code.

- In general, it's recommended to use local variables and pass them by reference  
whenever possible to promote modularity, maintainability, and readability.  

- `Only use global variables` when you have a good reason to do so, such as when  
dealing with `truly global state or constants`.